
# Linear Relaxation of Sigmoid

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
model = Sequential()
model.add(Activation('relu', input_dim=1))

In [3]:
import sys
sys.path.append('..')
import decomon
from decomon.models import convert, clone
from decomon import get_upper_box, get_lower_box, get_range_box

In [4]:
decomon_model = convert(model)

TypeError: to_monotonic() got an unexpected keyword argument 'grad_bounds'

In [5]:
def relu(x):
    return np.maximum(0., x)

In [6]:
def linear_hull_upper_decomon(lower, upper):
    x_min = lower*np.ones((1,1))
    x_max = upper*np.ones((1,1))
    box = np.concatenate([x_min[:,None], x_max[:,None]], 1)
    #print(box)
    output = decomon_model.predict([x_max, box])
    _, _, upper, _, w_u, b_u, _, _ = output

    return upper.min(), w_u.min(), b_u.min()

def linear_hull_lower_decomon(lower, upper):
    
    x_min = lower*np.ones((1,1))
    x_max = upper*np.ones((1,1))
    box = np.concatenate([x_min[:,None], x_max[:,None]], 1)
    
    output = decomon_model.predict([x_min, box])
    _, _, _, lower, _, _, w_l, b_l = output
    return lower.min(), w_l.min(), b_l.min()

In [12]:
def func(x_min, x_max):
    n_step=100
    if x_max <x_min:
        x_max_ = x_max + 0
        x_max = x_min
        x_min = x_max_
    x = np.linspace(x_min, x_max, n_step)
    y = np.linspace(-10, 10, 1000)
    
    _,w_u, b_u = linear_hull_upper_decomon(x_min,x_max)
    _, w_l, b_l = linear_hull_lower_decomon(x_min, x_max)
    s_x = relu(x)
    s_y = relu(y)
    z_x = w_u*x + b_u
    q_x = w_l*x + b_l

    plt.ylim(-2, 11)
    
    plt.plot(x, s_x, c='blue')
    plt.plot(x, z_x, c='orange')
    plt.plot(x, q_x, c='green' )

    plt.plot(y, s_y, '--', c='k')
    plt.title('Linear Relaxation of ReLU')

In [13]:
interact(func,
        x_min = widgets.FloatSlider(value=-10,
                                               min=-10,
                                               max=10.,
                                               step=0.01, continuous_update=False),
        x_max = widgets.FloatSlider(value=10,
                                               min=-10,
                                               max=10,
                                               step=0.01, continuous_update=False),
        )

interactive(children=(FloatSlider(value=-10.0, continuous_update=False, description='x_min', max=10.0, min=-10…

<function __main__.func(x_min, x_max)>